In [1]:
import glob
import json
import os

import anthropic

In [2]:
conversations = []

paths = glob.glob('./res/297.SNS 데이터 고도화/01-1.정식개방데이터/Validation/02.라벨링데이터/VL/*json')
target_count = 20
count = 0

for path in paths:
    with open(path, 'r') as f:
        conv_dict = json.load(f)
        if conv_dict['header']['dialogueInfo']['numberOfParticipants'] == 2:
            if conv_dict['header']['dialogueInfo']['numberOfUtterances'] > 30:
                conv_list = []
                for d in conv_dict['body']:
                    conv_list.append(d['participantID'] + ': ' + d['utterance'])
                if conv_list[0] == conv_list[1]:
                    print('Repeated Conversations')
                    continue
                conv_text = '\n'.join(conv_list)
                conversations.append(conv_text)
                count += 1
                if count == target_count:
                    break

target_count = 30
count = 0

for path in paths:
    with open(path, 'r') as f:
        conv_dict = json.load(f)
        if conv_dict['header']['dialogueInfo']['numberOfParticipants'] > 2:
            if conv_dict['header']['dialogueInfo']['numberOfUtterances'] > 30:
                conv_list = []
                for d in conv_dict['body']:
                    conv_list.append(d['participantID'] + ': ' + d['utterance'])
                if conv_list[0] == conv_list[1]:           
                    continue
                conv_text = '\n'.join(conv_list)
                conversations.append(conv_text)
                count += 1
                if count == target_count:
                    break

count

30

In [5]:
prompt = f"""You are given a conversation among three users below. Imagine what the users discussed prior to the conversation.
Please write the previous content, strictly following the tone and format of given conversation, with at least 3000 characters.

{conversations[-2]}
"""

In [6]:
print(conversations[-2])

P01: 웹툰이나 만화 자주 보는 편이야?
P02: 나 웹툰은 안 보고 만화는 코난이랑 짱구 슬램덩크 이런 거 좋아해~ 넌?
P03: 난 반대로 만화 안보고 웹툰만 엄청 봐 키키
P03: 요즘에는 시험 때문에 못 보기는 하는데 끝나면 다 몰아볼 예정이야 후욱후욱
P01: 키키 나는 둘 다 안 봄~
P01: 중학교 때는 가끔 만화책 빌려서 봐보긴 했는데 그것도 몇 개 안돼...
P01: 웹툰 재밌어?
P02: 어떤 작가 좋아해?
P02: 특별히 좋아하는 웹툰 있어?
P03: 이동건이라고 유미의 세포들 그린 작가인데 진짜 캐릭처도 귀엽고 감정선이 정말 여자인 건가 싶을 정도로 세심해요!
P03: 재미도 있고 지금도 뭐 신작 나왔더라고!
P01: 아 유미의 세포들! 들어보긴 했어~
P01: 유명하더라 영화인지 드라마로도 나오는 거 같던데? 김고은이 주인공으로
P02: 맞아 드라마 야나? 예고편 같은 거 떴던데~
P03: 웅웅 맞아 맞아 그거 웹툰이 엄청 흥행 해서 드라마화 한다고 하더라...
P03: 근데 개인적으로 웹툰 기반 드라마화 한 거 그렇게 재미는 없을 것 같아서 딱히 기대는 안 하는 중
P01: 맞아 전에 치즈인더트랩도... 별로 였다고 했던 거 같애
P02: 그건 드라마 영화 둘 다 이상해 ㅠㅠ 내용은 괜찮은데~
P03: 맞아 드라마로 했다가 평 안 좋았어서 영화로도 나왔는데 더 망했던 걸로 기억해 키키
P03: 그나마 요번에 한 간 떨어지는 동거 그거는 잘 된 것 같더라고?
P01: 아아 장기용이랑 혜리 나온 거? 뭐 구미호 얘기인 거야...?
P02: 아 진짜? 그것도 앞에만 보다 말았는데~
P03: 웅웅 맞아 맞아 장기용 여기도 나왔네 맞다 키키
P03: 혹시 마블 영화 좋아해?
P01: 보면 재밌는데 나는 막 챙겨보는 편은 아니야! 좋아해?
P02: 안 좋아해~ 다들 재밌다고 해서 맘 잡고 보려고 하는데 눈에 잘 안 들어오더라~ ㅠㅠ
P03: 약간 나랑 언니랑 취향 좀 다른 듯 키키
P03: 나는 마블 영화 엄청 챙겨 보는데 ㅠ
P01: 보면 재

In [7]:
client = anthropic.Anthropic(
    api_key=os.environ['CLAUDE_API_KEY']
)

message = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=4096,
    temperature=0.0,
    messages=[
        {"role": "user", "content": prompt}
    ]
)

message.usage  # API 호출 비용 대략 50원

/var/folders/25/r6dy_b_d6sb0y9nv_hk62c140000gn/T/ipykernel_81587/2185531226.py:5: DeprecationWarning: The model 'claude-3-5-sonnet-20240620' is deprecated and will reach end-of-life on October 22, 2025.
Please migrate to a newer model. Visit https://docs.anthropic.com/en/docs/resources/model-deprecations for more information.
  message = client.messages.create(


Usage(cache_creation=CacheCreation(ephemeral_1h_input_tokens=0, ephemeral_5m_input_tokens=0), cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=1415, output_tokens=4096, server_tool_use=None, service_tier='standard')

In [8]:
message.content[0].text.split('\n\n')

["Here's an imagined previous conversation among the three users, following the tone and format of the given conversation:",
 "P02: 야 너네 요즘 뭐하고 지내?\nP01: 나? 그냥 평소처럼 학교 다니고 공부하고 그러지 뭐\nP03: 나도 비슷해~ 요즘 시험기간이라 공부하느라 정신없어\nP02: 아 맞다 너네 시험기간이었지? 힘내!\nP01: 고마워~ 너는 뭐하고 지내?\nP02: 나는 요즘 드라마 보는 재미에 빠졌어\nP03: 오 무슨 드라마?\nP02: 요즘 '스물다섯 스물하나' 보고 있어. 진짜 재밌더라\nP01: 아 그거 남주혁이랑 김태리 나오는 거 맞지?\nP02: 응 맞아! 너도 알아?\nP01: 응 광고랑 포스터 같은 거 봤어. 근데 난 드라마를 잘 안 봐서...\nP03: 나도 들어본 것 같아! 근데 나는 요즘 공부하느라 드라마 볼 시간이 없어 ㅠㅠ\nP02: 아 그렇구나~ 시험 끝나면 한번 봐봐. 진짜 재밌어!\nP01: 난 드라마보다는 영화를 더 좋아하는 것 같아\nP03: 오 영화? 어떤 장르 좋아해?\nP01: 음... 로맨스 코미디나 액션 영화 주로 보는 것 같아\nP02: 나도 로맨스 코미디 좋아해! 최근에 본 거 중에 추천할 만한 거 있어?\nP01: 음... '너의 결혼식' 봤어? 그거 꽤 재밌던데\nP03: 아 박보영이랑 김영광 나오는 거? 나도 봤어! 진짜 재밌었지\nP02: 오 나 그거 아직 못 봤는데, 한번 봐야겠다!\nP01: 응 한번 봐봐~ 근데 너네는 영화관에서 보는 거 좋아해? 아니면 집에서 보는 게 좋아?\nP03: 나는 영화관! 대형 스크린이랑 음향이 주는 몰입감이 좋더라고\nP02: 나는 집에서 보는 게 더 좋아. 편하게 누워서 볼 수 있잖아\nP01: 아 그렇구나~ 나는 둘 다 좋아하는 편이야. 영화 종류에 따라 다른 것 같아\nP03: 맞아 영화 종류에 따라 다르지. 액션 영화는 영화관에서 봐야 제맛인 것 

In [9]:
lst = message.content[0].text.split('\n\n')[1:-1]
for l in lst[:25]:
    print(l)

In [10]:
for l in lst[25:50]:
    print(l)

In [11]:
print(len(message.content[0].text.split('\n\n')[1]))

3922


In [12]:
lst = message.content[0].text.split('\n\n')[1:-1]
lst

[]

In [13]:
import pickle

with open('./res/conv_long.pickle', 'wb') as f:
    pickle.dump(lst, f)